In [14]:
import geopandas as gpd
import requests
from shapely.geometry import box, Point, Polygon
from shapely.ops import unary_union

# Define the grid boundaries for Rome and grid size
ROME_BOUNDS = (12.531124, 41.86426, 12.535509, 41.86843)  # min_lon, min_lat, max_lon, max_lat
GRID_SIZE = 70 / 111320  # Approximate 70m in degrees (1 degree ~ 111.32 km)

# Function to create a 70x70m grid
def create_grid(bounds, grid_size):
    min_lon, min_lat, max_lon, max_lat = bounds
    lon_steps = int((max_lon - min_lon) / grid_size)
    lat_steps = int((max_lat - min_lat) / grid_size)

    polygons = []
    for i in range(lon_steps):
        for j in range(lat_steps):
            lon_min = min_lon + i * grid_size
            lat_min = min_lat + j * grid_size
            lon_max = lon_min + grid_size
            lat_max = lat_min + grid_size
            polygons.append(box(lon_min, lat_min, lon_max, lat_max))
    return gpd.GeoDataFrame({'geometry': polygons}, crs="EPSG:4326")


In [15]:
# Function to clean invalid geometries
def clean_geometries(gdf):
    # Check for invalid geometries
    invalid_geometries = gdf[~gdf.is_valid]
    if not invalid_geometries.empty:
        # print(f"Found invalid geometries: {invalid_geometries}")
        # Buffer with 0 to fix invalid geometries (snapping to valid)
        gdf['geometry'] = gdf['geometry'].buffer(0)
    return gdf


In [16]:
def calculate_proportions(grid, features):
    grid = grid.copy()

    for feature_name, feature_gdf in features.items():
        # Ensure the feature is in the same CRS as the grid
        feature_gdf = feature_gdf.to_crs(epsg=3395)

        if feature_name == 'trees':  # Trees are typically points
            # Count how many trees are within each grid cell
            grid[feature_name] = grid.geometry.apply(
                lambda cell: sum(cell.contains(point) for point in feature_gdf.geometry) / cell.area
            )

        elif feature_name in ['sealed_parking', 'gardens', 'parks', 'forests']:  # Handle other features
            # Check if feature is empty before proceeding
            if not feature_gdf.empty:
                grid[feature_name] = grid.geometry.apply(
                    lambda cell: feature_gdf.intersection(cell).area.sum() / cell.area if cell.area > 0 else 0
                )
            else:
                grid[feature_name] = 0  # If there are no features, set proportion to 0

        else:  # Handle polygons like buildings and roads
            # Check if feature is empty before proceeding
            if not feature_gdf.empty:
                grid[feature_name] = grid.geometry.apply(
                    lambda cell: feature_gdf.intersection(cell).area.sum() / cell.area if cell.area > 0 else 0
                )
            else:
                grid[feature_name] = 0  # If there are no features, set proportion to 0

    return grid

In [17]:
# Function to query Overpass API for specific OSM features
def query_overpass(tag, bounds):
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    (
        way[{tag}]({bounds[1]},{bounds[0]},{bounds[3]},{bounds[2]});
    );
    out body;
    >;
    out skel qt;
    """

    response = requests.get(overpass_url, params={'data': query})

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Error fetching data for {tag}: {response.status_code}")
        return gpd.GeoDataFrame()  # Return an empty GeoDataFrame if the request failed

    data = response.json()
    osm_data = data.get('elements', [])

    # Extract nodes
    node_map = {node["id"]: (node["lon"], node["lat"]) for node in osm_data if node["type"] == "node"}

    print(tag)

    # Extract tag and construct geometries
    polygons = []
    for element in osm_data:
        print(element)
        if "nodes" in element:
            coords = [node_map[node_id] for node_id in element["nodes"] if node_id in node_map]
            if len(coords) > 2:  # Ensure we have a valid polygon
                polygons.append(Polygon(coords))

    # Create a GeoDataFrame
    gdf = gpd.GeoDataFrame({'geometry': polygons}, crs="EPSG:4326")

    return gdf

In [18]:
# Create the grid
grid = create_grid(ROME_BOUNDS, GRID_SIZE)
# Query OSM for buildings, roads, sealed parking, parks, etc.
bounds = ROME_BOUNDS
buildings = query_overpass('building', bounds)
sealed_parking = query_overpass('amenity=parking', bounds)
roads = query_overpass('highway', bounds)
parks = query_overpass('leisure=park', bounds)
gardens = query_overpass('leisure=garden', bounds)
forests = query_overpass('landuse=forest', bounds)


building
{'type': 'way', 'id': 253732331, 'nodes': [2596192789, 2596192787, 2596192778, 2596192780, 2596192789], 'tags': {'building': 'yes'}}
{'type': 'way', 'id': 336308041, 'nodes': [3433615319, 3433615303, 3433615309, 3433615312, 3433615308, 3433615311, 3433615313, 3433615315, 3433615310, 3433615306, 3433615304, 3433615305, 3433615307, 3433615317, 3433615314, 3433615318, 3433615316, 3433615320, 3433615319], 'tags': {'building': 'yes', 'building:levels': '8'}}
{'type': 'way', 'id': 336315955, 'nodes': [3433648882, 3433648880, 3433648881, 3433648883, 3433648882], 'tags': {'building': 'yes', 'building:levels': '8'}}
{'type': 'way', 'id': 369375015, 'nodes': [3731691569, 3731691554, 3731691551, 3731691519, 3731691506, 3731691507, 3731691534, 3731691532, 3731691536, 3731691545, 3731691586, 3731691584, 3731691591, 3731691582, 3731691577, 3731691569], 'tags': {'building': 'yes', 'building:levels': '8'}}
{'type': 'way', 'id': 369375022, 'nodes': [3731691405, 3731691383, 3731691371, 37316913

In [19]:

trees = query_overpass('natural=tree', bounds)

natural=tree


In [20]:

# Reproject grid and features to a projected CRS for area calculation (e.g., EPSG:3395)
grid = grid.to_crs(epsg=3395)

# Clean feature geometries
buildings = clean_geometries(buildings)
sealed_parking = clean_geometries(sealed_parking)
roads = clean_geometries(roads)
parks = clean_geometries(parks)
gardens = clean_geometries(gardens)
forests = clean_geometries(forests)
trees = clean_geometries(trees)

# Combine and calculate proportions
features = {
    'buildings': buildings,
    'sealed_parking': sealed_parking,
    'roads': roads,
    'parks': parks,
    'gardens': gardens,
    'forests': forests,
    'trees': trees
}

grid_with_proportions = calculate_proportions(grid, features)

# Save the results to a file
grid_with_proportions.to_file("rome_land_use_proportions.geojson", driver="GeoJSON")

# Display results
print(grid_with_proportions[['buildings', 'sealed_parking', 'roads', 'parks', 'gardens', 'forests', 'trees']])


    buildings  sealed_parking     roads  parks  gardens  forests  trees
0    0.055432        0.060109  0.019791    0.0        0        0    0.0
1    0.000000        0.264484  0.010843    0.0        0        0    0.0
2    0.000000        0.062659  0.077622    0.0        0        0    0.0
3    0.106033        0.000000  0.025340    0.0        0        0    0.0
4    0.099654        0.025381  0.082663    0.0        0        0    0.0
5    0.115805        0.210297  0.158742    0.0        0        0    0.0
6    0.109562        0.145898  0.048269    0.0        0        0    0.0
7    0.000000        0.000000  0.033000    0.0        0        0    0.0
8    0.000000        0.000000  0.242535    0.0        0        0    0.0
9    0.000000        0.000000  0.005988    0.0        0        0    0.0
10   0.005287        0.341353  0.310866    0.0        0        0    0.0
11   0.010677        0.397984  0.231506    0.0        0        0    0.0
12   0.044554        0.000000  0.375033    0.0        0        0